# Supervised Fine-Tuning (SFT) with PEFT of Amazon Nova using Amazon SageMaker Training Job

In this notebook, we fine-tune LLM on Amazon SageMaker AI, using Python scripts and SageMaker ModelTrainer for executing a training job.


***

## Lab 1: Prerequisites

If you are going to use Sagemaker in a local environment. You need access to an IAM Role with the required permissions for Sagemaker. You can find [here](https://docs.aws.amazon.com/sagemaker/latest/dg/sagemaker-roles.html) more about it.

In [ ]:
import sagemaker
import boto3

sess = sagemaker.Session()
sagemaker_session_bucket = None

if sagemaker_session_bucket is None and sess is not None:
    # set to default bucket if a bucket name is not given
    sagemaker_session_bucket = sess.default_bucket()

try:
    role = sagemaker.get_execution_role()
except ValueError:
    iam = boto3.client("iam")
    role = iam.get_role(RoleName="sagemaker_execution_role")["Role"]["Arn"]

sess = sagemaker.Session(default_bucket=sagemaker_session_bucket)
bucket_name = sess.default_bucket()
default_prefix = sess.default_bucket_prefix

print(f"sagemaker role arn: {role}")
print(f"sagemaker bucket: {sess.default_bucket()}")
print(f"sagemaker session region: {sess.boto_region_name}")

***

## Lab 2: Prepare the dataset

We are going to load [HuggingFaceH4/Multilingual-Thinking](https://huggingface.co/datasets/HuggingFaceH4/Multilingual-Thinking) dataset

In [ ]:
from datasets import load_dataset

dataset = load_dataset(
    "HuggingFaceH4/Multilingual-Thinking", split="train"
)

dataset

In [ ]:
import pandas as pd

df = pd.DataFrame(dataset)

df.head()

In [ ]:
from sklearn.model_selection import train_test_split

train, temp = train_test_split(df, test_size=0.2, random_state=42)
val, test = train_test_split(temp, test_size=10, random_state=42)

print("Number of train elements: ", len(train))
print("Number of val elements: ", len(val))
print("Number of test elements: ", len(test))

### Utility functions

In [ ]:
import tiktoken
import json

model_id = "o200k_base"

def count_tokens(text):
    """Count number of tokens in a string"""
    encoding = tiktoken.get_encoding(model_id)
    return len(encoding.encode(text))


def count_message_tokens(messages_dict):
    """Count total tokens in the entire message structure"""
    # Convert the entire message structure to JSON string and count tokens
    message_json = json.dumps(messages_dict, ensure_ascii=False)
    return count_tokens(message_json)


def truncate_entire_message(messages, max_tokens=30000):
    """Truncate the entire message structure if it's still too long"""
    current_tokens = count_message_tokens(messages)

    if current_tokens <= max_tokens:
        return messages

    # Calculate proportional limits based on max_tokens
    think_limit = max(500, max_tokens // 8)  # ~12.5% for thinking
    content_limit = max(1000, max_tokens // 4)  # ~25% for regular content

    # Strategy: Remove or truncate the longest content sections first
    for message in messages.get("messages", []):
        if message.get("role") == "assistant":
            for content in message.get("content", []):
                if "text" in content:
                    text = content["text"]
                    # If this is a think section, truncate it more aggressively
                    if text.startswith("<think>"):
                        # Extract just the thinking part
                        think_start = text.find("<think>") + 7
                        think_end = text.find("</think>")
                        if think_end != -1:
                            thinking_content = text[think_start:think_end]
                            # Truncate thinking to smaller proportional size
                            truncated_thinking = truncate_message(
                                thinking_content, max_tokens=think_limit
                            )
                            content["text"] = f"<think>\n{truncated_thinking}\n</think>"
                        else:
                            # If malformed, just truncate the whole thing
                            content["text"] = truncate_message(
                                text, max_tokens=think_limit
                            )
                    else:
                        # Regular content, truncate with proportional limit
                        content["text"] = truncate_message(
                            text, max_tokens=content_limit
                        )

        # Check if we're under the limit now
        current_tokens = count_message_tokens(messages)
        if current_tokens <= max_tokens:
            break

    return messages


def truncate_message(message_str, max_tokens=20000):
    """Truncate the text based on token count, not character count"""
    current_tokens = count_tokens(message_str)

    if current_tokens <= max_tokens:
        return message_str

    print(f"Truncating message from {current_tokens} to ~{max_tokens} tokens...")

    # Get the encoding
    encoding = tiktoken.get_encoding(model_id)

    # Encode the text to tokens
    tokens = encoding.encode(message_str)

    # Truncate to max_tokens - 3 (to leave room for "...")
    truncated_tokens = tokens[: max_tokens - 3]

    # Decode back to text and add ellipsis
    truncated_text = encoding.decode(truncated_tokens) + "..."

    return truncated_text

Let's format the dataset by using the prompt style for Amazon Nova:

```
{
    "system": [{"text": Content of the System prompt}],
    "messages": [
        {
            "role": "user",
            "content": ["text": Content of the user prompt]
        },
        {
            "role": "assistant",
            "content": ["text": Content of the answer]
        },
        ...
    ]
}
```

In [ ]:
import textwrap

max_tokens = 3072
max_tokens_think = 2250

def prepare_dataset(sample):
    """Prepare dataset in the required format for Nova models"""
    messages = {"system": [], "messages": []}

    for el in sample["messages"]:
        if el["role"] == "system":
            system_prompt = """
            You are an AI assistant that thinks in {language} but responds in English.

            IMPORTANT: Follow this exact format for every response:
            1. First, write your reasoning and thoughts inside <think>...</think> tags
            2. Then, provide your final answer in English

            Always think through the problem in {language}, then translate your conclusion to English for the final response.
            """

            system_prompt = system_prompt.format(language=sample["reasoning_language"])
            system_prompt = textwrap.dedent(system_prompt).strip()

            messages["system"].append({"text": system_prompt.lower()})
        elif el["role"] == "user":
            messages["messages"].append(
                {"role": "user", "content": [{"text": el["content"].lower()}]}
            )
        else:
            if (
                el["thinking"] is not None
                and el["thinking"] != ""
                and el["thinking"] != "null"
            ):
                # Truncate thinking content by tokens, not characters
                reasoning_text = truncate_message(
                    el["thinking"].lower(), max_tokens=max_tokens_think
                )  # Leave room for other content
                messages["messages"].append(
                    {
                        "role": "assistant",
                        "content": [
                            {"text": f"<think>\n{reasoning_text}\n</think>"},
                            {"text": el["content"].lower()},
                        ],
                    }
                )
            else:
                content_text = truncate_message(
                    el["content"].lower(), max_tokens=max_tokens
                )
                messages["messages"].append(
                    {
                        "role": "assistant",
                        "content": [
                            {"text": content_text},
                        ],
                    }
                )

    # Check total token count and further truncate if needed
    total_tokens = count_message_tokens(messages)
    if total_tokens > max_tokens:
        print(
            f"Warning: Total message tokens ({total_tokens}) still too high, applying additional truncation..."
        )
        messages = truncate_entire_message(messages, max_tokens=max_tokens)

    return messages

In [ ]:
max_tokens = 3072
max_tokens_think = 2250

def prepare_dataset_test(sample):
    """Parse sample and format it for test dataset."""
    message = {
        "system": "",
        "query": "",
        "response": "",
    }

    for el in sample["messages"]:
        if el["role"] == "system":
            system_prompt = """
            You are an AI assistant that thinks in {language} but responds in English.

            IMPORTANT: Follow this exact format for every response:
            1. First, write your reasoning and thoughts inside <think>...</think> tags
            2. Then, provide your final answer in English

            Always think through the problem in {language}, then translate your conclusion to English for the final response.
            """

            system_prompt = system_prompt.format(language=sample["reasoning_language"])
            system_prompt = textwrap.dedent(system_prompt).strip()

            message["system"] = system_prompt.lower()
        elif el["role"] == "user":
            message["query"] = el["content"].lower()
        else:
            if (
                el["thinking"] is not None
                and el["thinking"] != ""
                and el["thinking"] != "null"
            ):
                reasoning_text = truncate_message(
                    el["thinking"].lower(), max_tokens=max_tokens_think
                )
                message["response"] = (
                    f"<think>\n{reasoning_text}\n</think>\n{el['content']}".lower()
                )
            else:
                message["response"] = truncate_message(
                    el["content"].lower(), max_tokens=max_tokens
                )

    return message

In [ ]:
from datasets import Dataset, DatasetDict
import json
from random import randint

train_dataset = Dataset.from_pandas(train)
val_dataset = Dataset.from_pandas(val)
test_dataset = Dataset.from_pandas(test)

dataset = DatasetDict({"train": train_dataset, "val": val_dataset, "test": test_dataset})

train_dataset = dataset["train"].map(
    prepare_dataset, remove_columns=list(dataset["train"].features)
)

print(json.dumps(train_dataset[randint(0, len(train_dataset) - 1)], indent=2))

val_dataset = dataset["val"].map(
    prepare_dataset, remove_columns=list(dataset["val"].features)
)

The validation dataset will be formatted with the structure below:

```
{
    "system": "Optional - String containing the system prompt, that sets the behavior, role, or personality of the model",
    "query": "String containing the input prompt",
    "response": "String containing the expected model output"
}
```

In [ ]:
import json

test_dataset = dataset["test"].map(
    prepare_dataset_test, remove_columns=list(dataset["test"].features)
)

print(json.dumps(test_dataset[randint(0, len(test_dataset) - 1)], indent=2))

### Upload to Amazon S3

In [ ]:
import boto3

In [ ]:
s3_client = boto3.client("s3")

if default_prefix:
    input_path = f"{default_prefix}/datasets/nova-sft-peft"
else:
    input_path = f"datasets/nova-sft-peft"

train_dataset_s3_path = f"s3://{bucket_name}/{input_path}/train/dataset.jsonl"
val_dataset_s3_path = f"s3://{bucket_name}/{input_path}/val/dataset.jsonl"
test_dataset_s3_path = f"s3://{bucket_name}/{input_path}/test/gen_qa.jsonl"

In [ ]:
import os

train_dataset.to_json("./data/train/dataset.jsonl")
val_dataset.to_json("./data/val/dataset.jsonl")
test_dataset.to_json("./data/test/gen_qa.jsonl")

s3_client.upload_file(
    "./data/train/dataset.jsonl", bucket_name, f"{input_path}/train/dataset.jsonl"
)

s3_client.upload_file(
    "./data/val/dataset.jsonl", bucket_name, f"{input_path}/val/dataset.jsonl"
)

s3_client.upload_file(
    "./data/test/gen_qa.jsonl", bucket_name, f"{input_path}/test/gen_qa.jsonl"
)

print(f"Training data uploaded to:")
print(train_dataset_s3_path)
print(val_dataset_s3_path)
print(test_dataset_s3_path)

***

## Lab 3: Model fine-tuning

We now define the PyTorch estimator to run the a Supervised fine-tuning (SFT) workload on the formatted tool-calling dataset for Amazon Nova models

In [ ]:
instance_type = "ml.g5.12xlarge"
instance_count = 1

instance_type

Let's define the container to execute the SFT workload for Amazon Nova models

In [ ]:
image_uri = f"708977205387.dkr.ecr.{sess.boto_region_name}.amazonaws.com/nova-fine-tune-repo:SM-TJ-SFT-ai-league"

image_uri

In [ ]:
recipe = "./recipes/nova_micro_1_0_g5_g6_12x_gpu_lora_sft.yaml"

In [ ]:
from sagemaker.pytorch import PyTorch

# define Training Job Name
job_name = f"train-nova-micro-sft-peft"

# define OutputDataConfig path
if default_prefix:
    output_path = f"s3://{bucket_name}/{default_prefix}/{job_name}"
else:
    output_path = f"s3://{bucket_name}/{job_name}"

epochs = 1

recipe_overrides = {
    "training_config": {
        "trainer": {
            "max_epochs": epochs,
        },
        "model": {
            "optim": {
                "lr": 2e-5,
                "weight_decay": 0.01,
                "sched": {"warmup_steps": 50},
            },
            "peft": {
                "peft_scheme": "lora",
                "lora_tuning": {
                    "alpha": 32,
                    "adapter_dropout": 0.1,
                },
            },
        },
    },
}

estimator = PyTorch(
    output_path=output_path,
    base_job_name=job_name,
    role=role,
    instance_count=instance_count,
    instance_type=instance_type,
    training_recipe=recipe,
    recipe_overrides=recipe_overrides,
    max_run=432000,
    sagemaker_session=sess,
    image_uri=image_uri,
    disable_profiler=True,
    debugger_hook_config=False,
    tags=[{"Key": "max_epochs", "Value": str(epochs)}],
)

In [ ]:
from sagemaker.inputs import TrainingInput

train_input = TrainingInput(
    s3_data=train_dataset_s3_path,
    distribution="FullyReplicated",
    s3_data_type="Converse",
)

val_input = TrainingInput(
    s3_data=val_dataset_s3_path,
    distribution="FullyReplicated",
    s3_data_type="Converse",
)

In [ ]:
# starting the train job with our uploaded datasets as input
estimator.fit(inputs={"train": train_input, "validation": val_input}, wait=False)

You can monitor the job directly from your notebook output. You can also refer the SageMaker AI console, which shows the status of the job and the corresponding CloudWatch logs for governance and observability, as shown in the following screenshots.